# Project1
## Momentum Trading
### MACD

In [4]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
#from MCForecastTools import MCSimulation
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
#from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import hvplot.pandas


%matplotlib inline

In [5]:
# Load .env enviroment variables
load_dotenv()

True

### Collect Investments Data Using Alpaca: `AAPLE` (stocks) 

In [6]:
# Set current amount of shares
my_apple = 1

In [7]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [8]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-10-30', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL"]

# Set timeframe to "1Minute" for Alpaca API
timeframe = "15Min"

# Get current closing prices for AAPL and SP500
df_tickers=alpaca.get_bars(
    tickers,
    timeframe,
    start=start_time,
    end=end_time
).df

df_tickers.head()

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-10-30 08:00:00+00:00,168.90,169.00,168.60,391,168.98,15303,168.871369,AAPL
2023-10-30 08:15:00+00:00,168.85,169.00,168.85,105,168.99,5249,168.955731,AAPL
2023-10-30 08:30:00+00:00,168.85,168.92,168.85,72,168.85,3911,168.863544,AAPL
2023-10-30 08:45:00+00:00,168.99,168.99,168.71,81,168.86,4184,168.869565,AAPL
2023-10-30 09:00:00+00:00,169.23,169.23,168.97,111,168.97,5241,169.137236,AAPL


In [9]:
# Reorganize the DataFrame
# Separate ticker data
df_tickers.drop(['high','low','trade_count','open','volume','vwap','symbol'], axis=1, inplace=True)

df_tickers.head()

,close
timestamp,
2023-10-30 08:00:00+00:00,168.90
2023-10-30 08:15:00+00:00,168.85
2023-10-30 08:30:00+00:00,168.85
2023-10-30 08:45:00+00:00,168.99
2023-10-30 09:00:00+00:00,169.23


### MACD formula

In [10]:
# Set MACD formula and iterate over Apple data for the specified intervals and period of time

moving_average_convergance_divergence_ema12 = df_tickers['close'].ewm(halflife=12).mean()
moving_average_convergance_divergence_ema26 = df_tickers['close'].ewm(halflife=26).mean()
  

In [11]:

MACD_line = moving_average_convergance_divergence_ema12 - moving_average_convergance_divergence_ema26
MACD_line.head()


timestamp
2023-10-30 08:00:00+00:00    0.000000
2023-10-30 08:15:00+00:00   -0.000389
2023-10-30 08:30:00+00:00   -0.000511
2023-10-30 08:45:00+00:00    0.001111
2023-10-30 09:00:00+00:00    0.005153
Name: close, dtype: float64

In [12]:
Signal_line = MACD_line.ewm(halflife=9).mean()
Signal_line.head()

timestamp
2023-10-30 08:00:00+00:00    0.000000
2023-10-30 08:15:00+00:00   -0.000202
2023-10-30 08:30:00+00:00   -0.000313
2023-10-30 08:45:00+00:00    0.000085
2023-10-30 09:00:00+00:00    0.001261
Name: close, dtype: float64

In [13]:
df_tickers=df_tickers.assign(ewm12=df_tickers['close'].ewm(halflife=12).mean())
df_tickers.head()

,close,ewm12
timestamp,,
2023-10-30 08:00:00+00:00,168.90,168.900000
2023-10-30 08:15:00+00:00,168.85,168.874278
2023-10-30 08:30:00+00:00,168.85,168.865714
2023-10-30 08:45:00+00:00,168.99,168.899527
2023-10-30 09:00:00+00:00,169.23,168.973469


In [14]:
df_tickers=df_tickers.assign(ewm26=df_tickers['close'].ewm(halflife=26).mean())
df_tickers.head()

,close,ewm12,ewm26
timestamp,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315


In [15]:
df_tickers=df_tickers.assign(macd=MACD_line)
df_tickers.head()

,close,ewm12,ewm26,macd
timestamp,,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000,0.000000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667,-0.000389
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224,-0.000511
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416,0.001111
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315,0.005153


In [16]:
df_tickers=df_tickers.assign(signal=Signal_line)
df_tickers.head()

,close,ewm12,ewm26,macd,signal
timestamp,,,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000,0.000000,0.000000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667,-0.000389,-0.000202
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224,-0.000511,-0.000313
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416,0.001111,0.000085
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315,0.005153,0.001261


In [17]:
df_tickers=df_tickers.assign(histogram=MACD_line-Signal_line)
df_tickers.head()

,close,ewm12,ewm26,macd,signal,histogram
timestamp,,,,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000,0.000000,0.000000,0.000000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667,-0.000389,-0.000202,-0.000187
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224,-0.000511,-0.000313,-0.000198
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416,0.001111,0.000085,0.001025
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315,0.005153,0.001261,0.003893


In [18]:
Close_plot = df_tickers['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Close_plot

:Curve   [timestamp]   (close)

In [19]:
MACD_plot = df_tickers['macd'].hvplot.line(    
    x='timestamp', 
    y='macd',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

In [20]:
Signal_plot = df_tickers['signal'].hvplot.line(    
    x='timestamp', 
    y='signal',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

In [21]:
plot = MACD_plot * Signal_plot 

plot

:Overlay
   .Curve.Macd   :Curve   [timestamp]   (macd)
   .Curve.Signal :Curve   [timestamp]   (signal)

In [22]:
Histogram_plot = df_tickers['histogram'].hvplot.bar(    
    x='timestamp', 
    y='histogram',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='MACD-Signal', 
    xlabel='timestamp',
    title='MACD Line - Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Histogram_plot

:Bars   [timestamp]   (histogram)

In [23]:
# itterating over MACD line and Signal Line values of the same date in the final Data Frame above
df_tickers['return']=df_tickers['close'].pct_change()
display(df_tickers)




,close,ewm12,ewm26,macd,signal,histogram,return
timestamp,,,,,,,
2023-10-30 08:00:00+00:00,168.9000,168.900000,168.900000,0.000000,0.000000,0.000000,NaN
2023-10-30 08:15:00+00:00,168.8500,168.874278,168.874667,-0.000389,-0.000202,-0.000187,-0.000296
2023-10-30 08:30:00+00:00,168.8500,168.865714,168.866224,-0.000511,-0.000313,-0.000198,0.000000
2023-10-30 08:45:00+00:00,168.9900,168.899527,168.898416,0.001111,0.000085,0.001025,0.000829
2023-10-30 09:00:00+00:00,169.2300,168.973469,168.968315,0.005153,0.001261,0.003893,0.001420
...,...,...,...,...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,169.891672,169.769723,0.121949,0.125345,-0.003396,-0.000177
2023-10-30 23:00:00+00:00,169.8200,169.887527,169.771369,0.116158,0.124658,-0.008500,-0.000294
2023-10-30 23:15:00+00:00,169.8500,169.885360,169.773928,0.111433,0.123669,-0.012236,0.000177


In [24]:
fund=10000
shares=fund/df_tickers.iloc[0,0]
print(f"Your Apple Shares are {shares}")

Your Apple Shares are 59.20663114268798


In [25]:
df_tickers["invest"]=df_tickers["close"] * shares
df_tickers.head()

,close,ewm12,ewm26,macd,signal,histogram,return,invest
timestamp,,,,,,,,
2023-10-30 08:00:00+00:00,168.90,168.900000,168.900000,0.000000,0.000000,0.000000,NaN,10000.000000
2023-10-30 08:15:00+00:00,168.85,168.874278,168.874667,-0.000389,-0.000202,-0.000187,-0.000296,9997.039668
2023-10-30 08:30:00+00:00,168.85,168.865714,168.866224,-0.000511,-0.000313,-0.000198,0.000000,9997.039668
2023-10-30 08:45:00+00:00,168.99,168.899527,168.898416,0.001111,0.000085,0.001025,0.000829,10005.328597
2023-10-30 09:00:00+00:00,169.23,168.973469,168.968315,0.005153,0.001261,0.003893,0.001420,10019.538188


In [26]:
df_tickers["X"]=np.where(df_tickers["macd"]>df_tickers["signal"],1,0) # Creating signals
df_tickers["X"]=df_tickers["X"].diff() # Finding long or short trades
Xreturns=df_tickers[df_tickers["X"].isin([1,-1])] # Extracting just the trades where 1 is long, -1 is short
Xreturns.reset_index(inplace=True)
trades = Xreturns[["X","timestamp","close","invest"]] # Dataframe with trade signals and dates
trades["sumclose"]=trades["close"].diff()
profitsbase=abs(trades["sumclose"]).sum()
trades["suminvest"]=trades["invest"].diff()
profitsshares=abs(trades["suminvest"]).sum()

C:\Users\samus\AppData\Local\Temp\ipykernel_25556\3017990232.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades["sumclose"]=trades["close"].diff()
C:\Users\samus\AppData\Local\Temp\ipykernel_25556\3017990232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades["suminvest"]=trades["invest"].diff()


In [27]:
trades.head()

,X,timestamp,close,invest,sumclose,suminvest
0,1.0,2023-10-30 08:45:00+00:00,168.9900,10005.328597,NaN,NaN
1,-1.0,2023-10-30 12:00:00+00:00,168.9300,10001.776199,-0.0600,-3.552398
2,1.0,2023-10-30 13:30:00+00:00,170.8939,10118.052102,1.9639,116.275903
3,-1.0,2023-10-30 22:45:00+00:00,169.8699,10057.424512,-1.0240,-60.627590


In [28]:
print(f"You made profit per share in MACD of ${profitsbase}")


You made profit per share in MACD of $3.0478999999999985


In [29]:
print(profitsshares)

180.4558910597989


In [30]:
macd_profit=fund+profitsshares

In [31]:
print(f"You made profit per investement in MACD of ${macd_profit}")

You made profit per investement in MACD of $10180.455891059799


In [32]:
X_plot = trades['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='Decission Making Points', 
    xlabel='timestamp',
    title='Long and Short Signals',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

X_plot

:Curve   [timestamp]   (close)

In [33]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-10-30', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["SPY"]

# Set timeframe to "1Minute" for Alpaca API
timeframe = "15Min"

# Get current closing prices for AAPL and SP500
df_spy=alpaca.get_bars(
    tickers,
    timeframe,
    start=start_time,
    end=end_time
).df

df_spy.head()

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-10-30 08:00:00+00:00,413.21,413.29,412.42,516,412.42,30412,413.138614,SPY
2023-10-30 08:15:00+00:00,413.20,413.38,412.98,503,413.22,23309,413.201941,SPY
2023-10-30 08:30:00+00:00,413.00,413.16,412.89,184,413.08,7853,413.064173,SPY
2023-10-30 08:45:00+00:00,412.99,412.99,412.66,138,412.85,13263,412.779127,SPY
2023-10-30 09:00:00+00:00,413.53,413.61,413.00,282,413.00,11376,413.319453,SPY


In [34]:
# Reorganize the DataFrame
# Separate ticker data
df_spy.drop(['high','low','trade_count','open','volume','vwap','symbol'], axis=1, inplace=True)

df_spy.head()

,close
timestamp,
2023-10-30 08:00:00+00:00,413.21
2023-10-30 08:15:00+00:00,413.20
2023-10-30 08:30:00+00:00,413.00
2023-10-30 08:45:00+00:00,412.99
2023-10-30 09:00:00+00:00,413.53


In [35]:
# Returns 
df_spy['return']=df_spy['close'].pct_change()
df_spy.head()


,close,return
timestamp,,
2023-10-30 08:00:00+00:00,413.21,NaN
2023-10-30 08:15:00+00:00,413.20,-0.000024
2023-10-30 08:30:00+00:00,413.00,-0.000484
2023-10-30 08:45:00+00:00,412.99,-0.000024
2023-10-30 09:00:00+00:00,413.53,0.001308


In [36]:
# Cumulative returns
df_spy['cumprod']=(1+df_spy['return']).cumprod()
display(df_spy.head)

<bound method NDFrame.head of                             close    return   cumprod
timestamp                                            
2023-10-30 08:00:00+00:00  413.21       NaN       NaN
2023-10-30 08:15:00+00:00  413.20 -0.000024  0.999976
2023-10-30 08:30:00+00:00  413.00 -0.000484  0.999492
2023-10-30 08:45:00+00:00  412.99 -0.000024  0.999468
2023-10-30 09:00:00+00:00  413.53  0.001308  1.000774
...                           ...       ...       ...
2023-10-30 22:45:00+00:00  415.32 -0.000169  1.005106
2023-10-30 23:00:00+00:00  415.34  0.000048  1.005155
2023-10-30 23:15:00+00:00  415.29 -0.000120  1.005034
2023-10-30 23:30:00+00:00  415.39  0.000241  1.005276
2023-10-30 23:45:00+00:00  415.18 -0.000506  1.004768

[64 rows x 3 columns]>

In [37]:

df_spy['invest']=df_spy['cumprod']*fund
display(df_spy)

,close,return,cumprod,invest
timestamp,,,,
2023-10-30 08:00:00+00:00,413.21,NaN,NaN,NaN
2023-10-30 08:15:00+00:00,413.20,-0.000024,0.999976,9999.757992
2023-10-30 08:30:00+00:00,413.00,-0.000484,0.999492,9994.917838
2023-10-30 08:45:00+00:00,412.99,-0.000024,0.999468,9994.675831
2023-10-30 09:00:00+00:00,413.53,0.001308,1.000774,10007.744246
...,...,...,...,...
2023-10-30 22:45:00+00:00,415.32,-0.000169,1.005106,10051.063624
2023-10-30 23:00:00+00:00,415.34,0.000048,1.005155,10051.547639
2023-10-30 23:15:00+00:00,415.29,-0.000120,1.005034,10050.337601


In [38]:
spy_plot = df_spy['invest'].hvplot.line(    
    x='timestamp', 
    y='invest',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='Cumulative Returns', 
    xlabel='timestamp',
    title='SPY Cumulative returns',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

spy_plot

:Curve   [timestamp]   (invest)

In [40]:
spy_profit=df_spy.iloc[-1,-1]
print(f"You made a profit of your SPY investement of $ {spy_profit}")

You made a profit of your SPY investement of $ 10047.675516081417


In [45]:
# AAPL RETURNS IF INVESTED AND DID NOT USE MACD
df_tickers_1 = df_tickers
df_tickers_1


,close,ewm12,ewm26,macd,signal,histogram,return,invest,X
timestamp,,,,,,,,,
2023-10-30 08:00:00+00:00,168.9000,168.900000,168.900000,0.000000,0.000000,0.000000,NaN,10000.000000,NaN
2023-10-30 08:15:00+00:00,168.8500,168.874278,168.874667,-0.000389,-0.000202,-0.000187,-0.000296,9997.039668,0.0
2023-10-30 08:30:00+00:00,168.8500,168.865714,168.866224,-0.000511,-0.000313,-0.000198,0.000000,9997.039668,0.0
2023-10-30 08:45:00+00:00,168.9900,168.899527,168.898416,0.001111,0.000085,0.001025,0.000829,10005.328597,1.0
2023-10-30 09:00:00+00:00,169.2300,168.973469,168.968315,0.005153,0.001261,0.003893,0.001420,10019.538188,0.0
...,...,...,...,...,...,...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,169.891672,169.769723,0.121949,0.125345,-0.003396,-0.000177,10057.424512,-1.0
2023-10-30 23:00:00+00:00,169.8200,169.887527,169.771369,0.116158,0.124658,-0.008500,-0.000294,10054.470101,0.0
2023-10-30 23:15:00+00:00,169.8500,169.885360,169.773928,0.111433,0.123669,-0.012236,0.000177,10056.246300,0.0


In [46]:
# Reorganize the DataFrame
# Separate ticker data
df_tickers_1.drop(['ewm12','ewm26','macd','signal','histogram','return','invest','X'], axis=1, inplace=True)

df_tickers_1.head()

,close
timestamp,
2023-10-30 08:00:00+00:00,168.90
2023-10-30 08:15:00+00:00,168.85
2023-10-30 08:30:00+00:00,168.85
2023-10-30 08:45:00+00:00,168.99
2023-10-30 09:00:00+00:00,169.23


In [47]:
# Returns 
df_tickers_1['return']=df_tickers_1['close'].pct_change()
df_tickers_1.head()

,close,return
timestamp,,
2023-10-30 08:00:00+00:00,168.90,NaN
2023-10-30 08:15:00+00:00,168.85,-0.000296
2023-10-30 08:30:00+00:00,168.85,0.000000
2023-10-30 08:45:00+00:00,168.99,0.000829
2023-10-30 09:00:00+00:00,169.23,0.001420


In [48]:
# Cumulative returns
df_tickers_1['cumprod']=(1+df_tickers_1['return']).cumprod()
display(df_tickers_1.head)

df_tickers_1['invest']=df_tickers_1['cumprod']*fund
display(df_tickers_1)

<bound method NDFrame.head of                               close    return   cumprod
timestamp                                              
2023-10-30 08:00:00+00:00  168.9000       NaN       NaN
2023-10-30 08:15:00+00:00  168.8500 -0.000296  0.999704
2023-10-30 08:30:00+00:00  168.8500  0.000000  0.999704
2023-10-30 08:45:00+00:00  168.9900  0.000829  1.000533
2023-10-30 09:00:00+00:00  169.2300  0.001420  1.001954
...                             ...       ...       ...
2023-10-30 22:45:00+00:00  169.8699 -0.000177  1.005742
2023-10-30 23:00:00+00:00  169.8200 -0.000294  1.005447
2023-10-30 23:15:00+00:00  169.8500  0.000177  1.005625
2023-10-30 23:30:00+00:00  169.8199 -0.000177  1.005446
2023-10-30 23:45:00+00:00  169.7000 -0.000706  1.004737

[64 rows x 3 columns]>

,close,return,cumprod,invest
timestamp,,,,
2023-10-30 08:00:00+00:00,168.9000,NaN,NaN,NaN
2023-10-30 08:15:00+00:00,168.8500,-0.000296,0.999704,9997.039668
2023-10-30 08:30:00+00:00,168.8500,0.000000,0.999704,9997.039668
2023-10-30 08:45:00+00:00,168.9900,0.000829,1.000533,10005.328597
2023-10-30 09:00:00+00:00,169.2300,0.001420,1.001954,10019.538188
...,...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,-0.000177,1.005742,10057.424512
2023-10-30 23:00:00+00:00,169.8200,-0.000294,1.005447,10054.470101
2023-10-30 23:15:00+00:00,169.8500,0.000177,1.005625,10056.246300


In [49]:
aapl_profit=df_tickers_1.iloc[-1,-1]
print(f"You made a profit of your AAPL investement of $ {aapl_profit}")

You made a profit of your AAPL investement of $ 10047.365304914154


In [41]:
values=[macd_profit,spy_profit]
labels=['MACD','SPY']
plot.pie(values=values, labesl=labels, title='MACD vs SPY Investement', autopct='%1.1f%%')

AttributeError: 'Overlay' object has no attribute pie.